# Decision Trees

<img src="https://media.giphy.com/media/hqUxZmsgSjeMqOkzh4/giphy.gif" width=250 align = right>

1. Decision trees are supervised models.
1. Can be used for Both classification/regression tasks.
1. They are simple tree like structure (hierarchical in nature).
1. Decision trees can be thought as nested if else conditions. 
1. Highly interpretable models, easy to explain the workings.

<br><br>



# Why Decision Trees are popular?
1. Easy to interpret and represent.
1. Mimic human level thought. tries to take decisions like a human does.
1. Ensemble models are made up of Decision trees that performs even better than individual Decision trees.
1. When feature are categorical, Decision Trees are preffered over other models.  

# Decision Tree Example

Q- Solve a Business Problem that wants to Predict whether a user clicks on an avdertisement of a car or not?

<img src="https://media.giphy.com/media/6swcfDQHr3UTm/giphy.gif" width=400 align = right>

<br><br>

| Sex      | Income | alreadyCar      | Techy | Age |
| ----------- | ----------- | ----------- | ----------- | ----------- |
| M      | <=5L       | Yes | Yes | 18-25
| F      | 5L-15L       | No | No | 25-50
|        | 15L+       |  |  | 50+


<br>
<img src="https://media.giphy.com/media/AO1rZwtl5qRJC/giphy.gif" width=200 align=left>
    
       Entropy is measure of randomness of a system.
     
     
<img src="./imgs/entropy.png" width=300> <br>

# CODE : Entropy 

In [1]:
import numpy as np

In [2]:
def entropy(var):
    N = var.shape[0]
    values, counts = np.unique(var, return_counts=True )
    
    ent = 0.0
    
    for i in counts:
        p = i/N
        ent += (p * np.log2(p))
        
    return -ent

In [3]:
Y = np.array([1,0,0,1,0,1,0,1])

In [4]:
X = np.array([1,1,1,1,1,1,1])

In [5]:
entropy(Y)

1.0

In [6]:
entropy(X)

-0.0

# Information Gain


<img src="https://media.giphy.com/media/8ZhxIppeXmBw606oUR/giphy.gif" width=300 align = right>

<br><br>
$$ Gain(Y, A)  = H_D(Y) - \sum_{i=1}^{k} \frac{|D_i|} {|D|}{ H_{D_i}(Y)} $$

# Golf Dataset

<img src="./imgs/golf.png" width=500 align = left>

# CODE : Split Data

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('golf.csv')

In [9]:
df.head()

,Outlook,Temperature,Humidity,Windy,Play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes


In [10]:
def divide_data(data, feature):
    # here we are working data frames.
    
    DATA = {}
    
    feat_values = list(data[feature].value_counts().index)
    occurence = list(data[feature].value_counts())
    
    for val in feat_values:
        DATA[val] = {'data' : pd.DataFrame([], columns = data.columns), 'len': 0}
        
        
    for ix in range(data.shape[0]):
        val = data[feature].iloc[ix]
        
        DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
        
        idx = feat_values.index(val)
        DATA[val]['len']  = occurence[idx]
    
    return DATA

In [11]:
divide_data(df, 'Outlook')

{'sunny': {'data':    Outlook Temperature Humidity  Windy Play
  0    sunny         hot     high  False   no
  1    sunny         hot     high   True   no
  7    sunny        mild     high  False   no
  8    sunny        cool   normal  False  yes
  10   sunny        mild   normal   True  yes,
  'len': 5},
 'rainy': {'data':    Outlook Temperature Humidity  Windy Play
  3    rainy        mild     high  False  yes
  4    rainy        cool   normal  False  yes
  5    rainy        cool   normal   True   no
  9    rainy        mild   normal  False  yes
  13   rainy        mild     high   True   no,
  'len': 5},
 'overcast': {'data':      Outlook Temperature Humidity  Windy Play
  2   overcast         hot     high  False  yes
  6   overcast        cool   normal   True  yes
  11  overcast        mild     high   True  yes
  12  overcast         hot   normal  False  yes,
  'len': 4}}

# CODE : Information Gain

In [12]:
def information_gain(data, feature):
    examples = data.shape[0]
    
    DATA = divide_data(data, feature)
    
    keys = DATA.keys()
    
    
    ent_of_children = 0.0
    
    for key in keys:
        ent_of_children +=  ( (DATA[key]['len']/examples) * entropy(DATA[key]['data']['Play']) )
        
    info_gain = entropy(data['Play']) - ent_of_children
    return info_gain

In [13]:
information_gain(df, 'Outlook')

0.24674981977443933

In [14]:
information_gain(df, 'Windy')

0.04812703040826949

In [15]:
information_gain(df, 'Temperature')

0.02922256565895487

In [16]:
information_gain(df, 'Humidity')

0.15183550136234159

# Constructing a Decision Tree

<img src="./imgs/tree.png" width=500>

<img src="https://media.giphy.com/media/cKKXNlTYino7hWNXwl/giphy.gif" width=300 align = right>
<br><br><br>

# Stopping Condition
1. Pure Node
2. Can't grow the tree anymore because of lack of points.
3. If already reach a max depth.

# CODE : Building a Decision Tree

In [73]:
class DecisionTree:
    
    # constructor
    def __init__(self, depth=0, max_depth=5):
        # Creating a Node
        self.children = {}
        self.fkey = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
        
    def train(self, data):
        features = ['Outlook', 'Temperature', 'Humidity', 'Windy']
        
        info_gains = []
        
        for f in features:
            i_gain = information_gain(data, f)
            info_gains.append(i_gain)
            
        # finding the best feature
        self.fkey = features[np.argmax(info_gains)]
        
        #Spliting the Data
        DATA = divide_data(data, self.fkey)
        
        
        
        # Giving a target label to the Node
        labels = list(data['Play'].value_counts().index)
        freq = list(data['Play'].value_counts().values)
        
        self.target = labels[np.argmax(freq)]
        
        
    
        
        ###### STOPPING CONDITIONS ######
        
        have_data = 0
        keys = DATA.keys()
        
        for key in keys:
            if DATA[key]['len'] > 0:
                have_data +=1
        
        # 1. If it is pure node 
        if have_data<2:
            return 
        
    
        # 2. Early Stop if you have reached max depth
        if(self.depth >= self.max_depth):
            return
        
        
        print("\t"*self.depth + "Making tree with - ", self.fkey)
        
        
        # Recursively train child Node
        for key in keys:
            new_data = DATA[key]['data']
            self.children[key] = DecisionTree(depth = self.depth + 1)
            self.children[key].train(new_data)
            
        return  
    
    
    def predict(self, test):
        if self.children == {}:
            return self.target
        return self.children[test[self.fkey][0]].predict(test)

# Explore the Model

In [74]:
model = DecisionTree()

In [75]:
model.train(df)

Making tree with -  Outlook
	Making tree with -  Humidity
	Making tree with -  Windy


In [76]:
model

In [77]:
model.target

'yes'

In [78]:
model.fkey

'Outlook'

In [79]:
model.children

{'sunny': <__main__.DecisionTree at 0x7ff5d0f565e0>,
 'rainy': <__main__.DecisionTree at 0x7ff5d0eebf70>,
 'overcast': <__main__.DecisionTree at 0x7ff5d0abfb50>}

In [80]:
model.children['sunny']

In [81]:
model.children['sunny'].fkey

'Humidity'

In [82]:
model.children['sunny'].children

{'high': <__main__.DecisionTree at 0x7ff5d0ad6280>,
 'normal': <__main__.DecisionTree at 0x7ff5d0ed82e0>}

In [83]:
model.children['sunny'].children['high'].children

{}

In [84]:
model.children['overcast'].target

'yes'

In [85]:
model.children['overcast'].children

{}

# Prediction Time

<img src="https://media.giphy.com/media/UTpbmJmHzOOApoGRuW/giphy.gif" width=200 align=left>

In [94]:
x_test = pd.DataFrame([['sunny', 'hot', 'normal', False]], columns=list(df.columns.values[:-1]))

In [95]:
x_test

,Outlook,Temperature,Humidity,Windy
0,sunny,hot,normal,False


In [96]:
model.predict(x_test)

'yes'